<a href="https://colab.research.google.com/github/kimberlyhau/StockMarketPredictor/blob/main/Final_LSTMModel_Predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [128]:
#imports
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import re
import os
from google.colab import drive
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import plotly.express as px

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [129]:
#file paths
sentiment_file = os.path.join('/content/drive/My Drive/sentiments.txt')
sentiment_file_2 = os.path.join('/content/drive/My Drive/sentimentsNY_final.txt')
stock_prices_folder = os.path.join('/content/drive/My Drive/stock_market_data')

In [130]:
#open cnn and ny times sentiment data
with open(sentiment_file, 'r') as file_obj:
    text = file_obj.read()
    txt_list = text.split('[')

with open(sentiment_file_2, 'r') as file_obj:
    text = file_obj.read()
    txt_list_ny = text.split('[')

In [131]:
#open stock prices
stock_data_lst = []
for filename in os.listdir(stock_prices_folder):
    if ".txt" in filename:
      #file_object = open(folder + "/" + filename, "r")
      with open(stock_prices_folder + "/" + filename, 'r') as file_obj:
        text = file_obj.read()
        stock_data_lst.append([filename.replace('.txt', ''), re.split(',|\n', text)])

In [ ]:
#organize stock prices
stock_data = []
for stocks in stock_data_lst:
  #stocks_df = pd.DataFrame(columns = ['Stock Price', 'Datestamp'])
  stock_name = stocks[0]
  print (stock_name)
  datestamp_lst = []
  stock_price_lst = []
  if stock_name != "SP500Data":
    for stock_elem in stocks[1]:
      if re.search('[a-zA-Z]', stock_elem):
        continue
      elif '-' in stock_elem:
        elems = stock_elem.split('-')
        datestamp = int(elems[0])*10000 + int(elems[1])*100 + int(elems[2])
        datestamp_lst.append(datestamp)
      elif '.' in stock_elem:
        stock_price = float(stock_elem)
        stock_price_lst.append(stock_price)
    stock_data.append([stock_name, pd.DataFrame(list(zip(stock_price_lst, datestamp_lst)), columns =['Stock Price', 'Datestamp'])])

In [133]:
#organize CNN data
datestamp_lst = []
sentiment_lst = []
date_lst = []
for i in range(1, len(txt_list)):
  elems = re.split("'|-|T|:|Z|,|]| ", txt_list[i])
  while("" in elems) :
    elems.remove("")
  date = int(elems[0])*10000 + int(elems[1])*100 + int(elems[2])
  datestamp = int(elems[0])*10000000000 + int(elems[1])*100000000 + int(elems[2])*1000000 + int(elems[3])*10000 + int(elems[4])*100 + int(elems[5])
  sentiment = float(elems[6])  
  # datestamp_lst.append(datestamp) datestamp_lst
  sentiment_lst.append(sentiment)
  date_lst.append(date)
sentiment_df = pd.DataFrame(list(zip(sentiment_lst, date_lst)), columns =['Sentiment', 'Date'])

In [ ]:
#clean up cnn data
sentiment_df_sorted = sentiment_df.sort_values('Date').set_index('Date').fillna(0)
sentiment_df_sorted =  (sentiment_df_sorted.groupby(level='Date').mean())
sentiment_df_sorted['null'] = [0.0]*len(sentiment_df_sorted)

print (sentiment_df_sorted)

In [135]:
#organize NY times data
datestamp_lst = []
sentiment_lst = []
date_lst = []
for i in range(1, len(txt_list_ny)):
  elems = re.split("'|-|T|:|Z|,|]| ", txt_list_ny[i])
  while("" in elems) :
    elems.remove("")
  if elems[0] == 'imestamp(':
    elems = elems[1:]
  date = int(elems[0])*10000 + int(elems[1])*100 + int(elems[2])
  
  datestamp = int(elems[0])*10000000000 + int(elems[1])*100000000 + int(elems[2])*1000000  #+ int(elems[3])*10000 + int(elems[4])*100 + int(elems[5])
  sentiment = float(elems[-1])  
  #  datestamp_lst.append(datestamp) datestamp_lst
  sentiment_lst.append(sentiment)
  date_lst.append(date)
sentiment_df_ny = pd.DataFrame(list(zip(sentiment_lst, date_lst)), columns =['Sentiment', 'Date'])

In [ ]:
#organize ny times data
sentiment_df_sorted_ny = sentiment_df_ny.sort_values('Date').set_index('Date').fillna(0)
sentiment_df_sorted_ny =  (sentiment_df_sorted_ny.groupby(level='Date').mean())
sentiment_df_sorted_ny['null'] = [0.0]*len(sentiment_df_sorted_ny)
(sentiment_df_sorted_ny)

In [137]:
#LSTM Model
input_features =2
hidden = 20
class Model(nn.Module):
    
    def __init__(self):
        super(Model, self).__init__()
        self.linear_1 = nn.Linear(input_features, hidden) #first linear layer
        self.relu = nn.ReLU() #relu layer
        self.lstm = nn.LSTM (hidden ,  hidden_size= hidden, num_layers = 1,) #LSTM
        self.linear = nn.Linear(hidden,1) #output layer
 
    def forward(self,x):
      x = self.linear_1(x)
      x = self.relu(x).reshape(1,1,20)

      lstm_out, x = self.lstm(x) #.reshape(20,1)
      out = self.linear(lstm_out)
      
      return out #.reshape(1,1)




In [ ]:
#Train Modle
#master lists
masterNames = []
masterTrainAcc = []
masterValidAcc = []
masterTestAcc = []

#Need to run for each of the 11 sectors
for k in range(0,len(stock_data)):
  #get stock prices
  temp_stock =  (stock_data[k][1][252:]).reset_index(drop = True).set_index('Datestamp')


  #store and print which one is in use
  print (stock_data[k][0][:-9])
  masterNames.append(stock_data[k][0][:-9])

  ##prepare data
  temp_stock =  (stock_data[k][1][252:]).reset_index(drop = True).set_index('Datestamp') #get correspondinging 

  #combine sentiment and stock prices  
  sentiment_df_sorted_ny['Stock Price'] = temp_stock.copy().dropna()
  sentiment_df_sorted_ny = sentiment_df_sorted_ny.dropna()
  sentiment_df_sorted_ny['CNN Sentiment'] = sentiment_df_sorted["Sentiment"].copy().fillna(0)
  sentiment_df_sorted_ny['CNN null'] = sentiment_df_sorted["null"].copy().fillna(0)

  MasterDf = sentiment_df_sorted_ny.copy()
  MasterDf = MasterDf.fillna(0)
  MasterDf['Sentiment'] = MasterDf[["Sentiment", "CNN Sentiment"]].values.tolist()
  MasterDf['null'] = MasterDf[["null", "CNN null"]].values.tolist()
  MasterDf = MasterDf.fillna(0)

  #Split the data into train nad test set
  train = (MasterDf[:int(MasterDf.shape[0]*0.8)]).sample(frac=1)

  X_train = torch.tensor(train['null'].reset_index(drop = True))
  y_train = torch.tensor(train['Stock Price'].reset_index(drop = True))

  test_and_val = (MasterDf[int(MasterDf.shape[0]*0.8):]).sample(frac=1)

  X_test_df = test_and_val[:int(test_and_val.shape[0]*0.5)]['null']
  y_test_df = test_and_val[:int(test_and_val.shape[0]*0.5)]['Stock Price']

  X_test = torch.tensor(X_test_df.reset_index(drop = True))
  y_test = torch.tensor(y_test_df.reset_index(drop = True))

  X_val_df = test_and_val[int(test_and_val.shape[0]*0.5):]['null']
  y_val_df = test_and_val[int(test_and_val.shape[0]*0.5):]['Stock Price']

  X_val = torch.tensor(X_val_df.reset_index(drop = True))
  y_val = torch.tensor(y_val_df.reset_index(drop = True))

  #Model Params
  epochs = 40
  net = Model()
  learning_rate = 0.05
  criterion = nn.MSELoss()# BCE??
  optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate)

  #lists to store
  train_loss_epoch = []
  train_acc_epoch = []

  valid_acc_epoch = []
  valid_loss_epoch = []

  model_y_valid = []
  model_y = []

  #training
  for epoch in range(epochs):
    train_acc = [] 
    train_loss =  []  
    valid_loss = []   
    acc_valid = [] 

    #update model weights
    for i in range(0,len(X_train)):
      optimizer.zero_grad()

      y = net(X_train[i].float().view(1,2))
      loss = criterion (y, y_train[i].float().view(1,1))
      
      loss.backward()
      optimizer.step()

      train_acc.append((np.abs(y.item() - y_train[i].float().view(1,1).numpy())[0][0])/y_train[i].float().view(1,1).numpy()[0][0]) #sccuracy
      train_loss.append(loss.item()) #loss

      #store model values on last epoch to plot
      if epoch == 39:
        model_y.append(y.item())

    for j in range(0,len(X_val)):

      y_2 = net(X_val[j].float().view(1,2))
      loss_v = criterion (y_2, y_val[j].float().view(1,1))

      acc_valid.append((np.abs(y_2.item() - y_val[j].float().view(1,1).numpy())[0][0])/y_val[j].float().view(1,1).numpy()[0][0])
      valid_loss.append(loss_v.item())

      #store model values on last epoch to plot
      if epoch == 39:
        model_y_valid.append(y_2.item())
      

    #Store average accuracy and loss for plotting
    train_loss_epoch.append(np.mean(train_loss))
    train_acc_epoch.append(np.mean(train_acc))      
    valid_acc_epoch.append(np.mean(acc_valid))
    valid_loss_epoch.append(np.mean(valid_loss))
  
    #print results
    print('Epoch: %04d  Training Loss: %.2f Training Error: %.2f Valid Loss: %.2f Valid Error: %.2f' % (epoch + 1, np.mean(train_loss), np.mean(train_acc)*100,np.mean(valid_loss), np.mean(acc_valid)*100))

  #store overall results to display after training everything        
  masterTrainAcc.append(np.mean(train_acc)*100)
  masterValidAcc.append(np.mean(acc_valid)*100)

  ##Plots
  #epoch/loss
  plt.title(stock_data[k][0][:-9] + " Epoch vs Loss")
  plt.plot(train_loss_epoch, label="Train")
  plt.xlabel("Epoch")
  plt.ylabel("Loss")
  plt.legend(loc='best')
  plt.grid()
  plt.show()

  #epoch/errror
  plt.title(stock_data[k][0][:-9]+" Epoch vs Error")
  plt.plot(train_acc_epoch, label="Train")
  plt.xlabel("Epoch")
  plt.ylabel("Error (%)")
  plt.legend(loc='best')
  plt.grid()
  plt.show()

  #Train results
  plt.title("Train: "+stock_data[k][0][:-9]+" Model Close Price vs True Close Price")
  plt.plot([i for i in range(0,len(y_train.detach().numpy()))], y_train.detach().numpy()) #, label="Train Price")
  plt.plot([i for i in range(0,len(y_train.detach().numpy()))], model_y, label="Model Price")
  plt.xticks([i for i in range(0,len(y_train.detach().numpy()), 40)],pd.to_datetime(train.index.sort_values (), format = "%Y%m%d")[::40],rotation = 45)
  plt.xlabel("Date")
  plt.ylabel("Stock Price")
  plt.legend(loc='best')
  plt.grid()
  plt.show()

  #Validation results
  plt.title("Valid :" +stock_data[k][0][:-9]+" Model Close Price vs True Close Price")
  plt.plot([i for i in range(0,len(y_val.detach().numpy()))], y_val.detach().numpy(), label="Valid Close Price")
  plt.plot([i for i in range(0,len(y_val.detach().numpy()))], model_y_valid, label="Model Close Price")
  plt.xticks([i for i in range(0,len(y_val.detach().numpy()),10)],pd.to_datetime(X_val_df.index.sort_values (), format = "%Y%m%d")[::10],rotation = 45)
  plt.xlabel("Date")
  plt.ylabel("Stock Price")
  plt.legend(loc='best')
  plt.grid()
  plt.show()


  ##test
  #Get test results
  model_y = []
  test_acc = []
  test_loss = []

  for i in range(0,len(X_test)):
 
      y = net(X_test[i].float().view(1,2))
      loss = criterion (y, y_test[i].float().view(1,1))

  
      test_acc.append((np.abs(y.item() - y_test[i].float().view(1,1).numpy())[0][0])/y_test[i].float().view(1,1).numpy()[0][0])
      test_loss.append(loss.item())

      model_y.append(y.item())

  
  #PLot Test Results   
  plt.title("Test: "+stock_data[k][0][:-9]+" Model Close Price vs True Close Price")
  plt.plot([i for i in range(0,len(y_test.detach().numpy()[:len(y_test)-1]))], y_test.detach().numpy()[:len(y_test)-1], label="Train Price")
  plt.plot([i for i in range(0,len(y_test.detach().numpy()[:len(y_test)-1]))], model_y[1:], label="Model Price")
  plt.xticks([i for i in range(0,len(y_test.detach().numpy()[:len(y_test)-1]))][::5],pd.to_datetime(X_test_df[1:].index.sort_values(), format = "%Y%m%d")[::5],rotation = 45)
  plt.xlabel("Date")
  plt.ylabel("Stock Price")
  plt.legend(loc='best')
  plt.grid()
  plt.show()

  masterTestAcc.append(np.mean(test_acc)*100)

 
                

In [ ]:
result =  (pd.DataFrame({'Sector':masterNames, 'Training Error (%)':masterTrainAcc,'Test Error (%)':masterTestAcc}))
display (result)
print (result.mean(axis=0))
#'Validation Accuracy':masterValidAcc,
